In [ ]:
import os
import cv2
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader

# Dataset 클래스 정의
class DeepFashion2Dataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(img_dir)
        self.masks = os.listdir(mask_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])

        # Load image and mask
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Transform
        if self.transform:
            image = self.transform(image)
            mask = T.ToTensor()(mask)

        return image, mask

# 데이터 경로 설정
img_dir = '/path/to/deepfashion2/images'
mask_dir = '/path/to/deepfashion2/masks'

# Transform 정의
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((512, 512)),
    T.ToTensor()
])

# Dataset 및 DataLoader
dataset = DeepFashion2Dataset(img_dir, mask_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
from torchvision.models.segmentation import deeplabv3_resnet50

# DeepLabV3+ 모델 불러오기
model = deeplabv3_resnet50(pretrained=False, num_classes=2)  # 2 클래스: 배경과 의상

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [ ]:
import torch.nn as nn
import torch.optim as optim

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for images, masks in dataloader:
        images, masks = images.to(device), masks.long().to(device)

        # Forward
        outputs = model(images)['out']
        loss = criterion(outputs, masks)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(dataloader):.4f}")


In [ ]:
import matplotlib.pyplot as plt

# 모델 평가 모드
model.eval()

with torch.no_grad():
    for images, masks in dataloader:
        images = images.to(device)
        outputs = model(images)['out']
        preds = torch.argmax(outputs, dim=1).cpu()

        # 시각화
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 3, 1)
        plt.title("Input Image")
        plt.imshow(images[0].cpu().permute(1, 2, 0))

        plt.subplot(1, 3, 2)
        plt.title("Ground Truth")
        plt.imshow(masks[0].cpu(), cmap='gray')

        plt.subplot(1, 3, 3)
        plt.title("Prediction")
        plt.imshow(preds[0], cmap='gray')

        plt.show()
        break


In [ ]:
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((512, 512)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(15),
    T.ColorJitter(brightness=0.2, contrast=0.2),
    T.ToTensor()
])
